In [1]:
suppressPackageStartupMessages(library(plm))
suppressPackageStartupMessages(library(nlme))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(lme4))
suppressPackageStartupMessages(library(repr))
suppressPackageStartupMessages(library(dplyr))


Warning message:
"package 'plm' was built under R version 3.6.2"


In [2]:
data_train <- read_csv('../data/train_processed.csv')
data_valid <- read_csv('../data/valid_processed.csv')

Parsed with column specification:
cols(
  .default = col_double(),
  income_class = col_character(),
  density_class = col_character(),
  climate = col_character()
)

See spec(...) for full column specifications.

Parsed with column specification:
cols(
  .default = col_double(),
  income_class = col_character(),
  density_class = col_character(),
  climate = col_character()
)

See spec(...) for full column specifications.



In [3]:
head(data_train)

month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,monthly_count_ramp,...,monthly_ws_light_br,monthly_ws_gentle_br,monthly_ws_moderate_br,historic_ws_calm,historic_ws_light_air,historic_ws_light_br,historic_ws_gentle_br,historic_ws_moderate_br,avg_fertility_rate,unacast_session_count
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
5,2018,0,0,0,0,0.0,0,0.00,0,...,0,0,0,0,0,0,0,0,63.90687,190
4,2019,0,0,0,0,0.0,0,0.00,0,...,0,0,0,1,18,63,41,54,61.17312,7
6,2018,0,0,0,0,0.0,0,0.00,0,...,0,0,0,0,2,2,2,3,65.72500,32
9,2019,0,0,0,0,0.0,0,0.00,0,...,0,0,0,0,6,7,4,5,65.12750,2332
7,2019,176,16,160,1540398,438295.5,156642,56107.95,0,...,54,58,35,0,13,50,23,20,58.47000,156
9,2019,0,0,0,0,0.0,0,0.00,0,...,0,0,0,0,0,1,0,0,62.94125,3175


In [19]:
colnames(data_train)[1]

[1] "month"

In [23]:
unique(data_train['density_class'])

density_class
<chr>
LD
MD
HD


In [69]:
unique(data_train['monthly_rain'])

monthly_rain
<dbl>
0


In [6]:
data_train_numeric <- select(data_train, -income_class, -climate, -density_class, -month, -year, -unacast_session_count)
dim(data_train_numeric)

[1] 39592   621

In [7]:
scaled_data <- scale(data_train_numeric, scale= TRUE)

In [27]:
data_train_2 <- cbind(data_train[c('month','year','density_class', 'unacast_session_count')], scaled_data)
dim(data_train_2)

[1] 39592   625

In [61]:
data_train_2 <- select(data_train_2, - monthly_snow)

In [66]:
col_names <- colnames(data_train_2) 
x <- col_names[1]
for (i in seq(2,length(col_names)-587)){
    
    if (col_names[i] == 'density_class' | col_names[i] == 'unacast_session_count'){
        next
    }
    else{
        x <- paste(x, " + ", col_names[i])   
    }
}
x

[1] "month  +  year  +  monthly_number_of_sessions  +  monthly_unique_sessions  +  monthly_avg_length_of_session  +  monthly_avg_light_activity  +  monthly_avg_moderate_activity  +  monthly_avg_vigorous_activity  +  monthly_count_ramp  +  monthly_count_zipline  +  monthly_count_spinner  +  monthly_count_pull_under  +  monthly_under_40  +  monthly_temp_40_to_50  +  monthly_temp_50_to_60  +  monthly_temp_60_to_70  +  monthly_temp_70_to_80  +  monthly_temp_80_to_90  +  monthly_over_90  +  monthly_avg_length_under_40  +  monthly_avg_length_temp_40_to_50  +  monthly_avg_length_temp_50_to_60  +  monthly_avg_length_temp_60_to_70  +  monthly_avg_length_temp_70_to_80  +  monthly_avg_length_temp_80_to_90  +  monthly_avg_length_over_90  +  monthly_cloudy  +  monthly_clear  +  monthly_foggy  +  monthly_avg_cloudy_length  +  monthly_avg_clear_length  +  monthly_avg_foggy_length"

In [67]:


model_intercept <- glmer(paste("unacast_session_count ~ ", x, " + (1 | density_class)"), 
                        data = data_train_2)

# model_slope <- lmer(inv ~ value + capital + (value | firm), 
#                     data = data_train)
summary(model_intercept)

Warning message in glmer(paste("unacast_session_count ~ ", x, " + (1 | density_class)"), :
"calling glmer() with family=gaussian (identity link) as a shortcut to lmer() is deprecated; please call lmer() directly"

Correlation matrix not shown by default, as p = 33 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Linear mixed model fit by REML ['lmerMod']
Formula: unacast_session_count ~ month + year + monthly_number_of_sessions +  
    monthly_unique_sessions + monthly_avg_length_of_session +  
    monthly_avg_light_activity + monthly_avg_moderate_activity +  
    monthly_avg_vigorous_activity + monthly_count_ramp + monthly_count_zipline +  
    monthly_count_spinner + monthly_count_pull_under + monthly_under_40 +  
    monthly_temp_40_to_50 + monthly_temp_50_to_60 + monthly_temp_60_to_70 +  
    monthly_temp_70_to_80 + monthly_temp_80_to_90 + monthly_over_90 +  
    monthly_avg_length_under_40 + monthly_avg_length_temp_40_to_50 +  
    monthly_avg_length_temp_50_to_60 + monthly_avg_length_temp_60_to_70 +  
    monthly_avg_length_temp_70_to_80 + monthly_avg_length_temp_80_to_90 +  
    monthly_avg_length_over_90 + monthly_cloudy + monthly_clear +  
    monthly_foggy + monthly_avg_cloudy_length + monthly_avg_clear_length +  
    monthly_avg_foggy_length + (1 | density_class)
   Data: data_train

In [42]:
?glmer